In [1]:
from bnlp import NLTKTokenizer
import pandas as pd
import pickle
import numpy as np
from gensim.models import Phrases
from gensim.models.phrases import Phraser
import warnings
warnings.filterwarnings('ignore')

c:\users\administrator\appdata\local\programs\python\python38\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
df = pd.read_csv("bangla_news.csv")
pd.set_option("display.max_colwidth",-1)

fobj = open("recom.pkl","rb")
lda = pickle.load(fobj)
fobj.close()

fileob = open("dtm.pkl","rb")
token = pickle.load(fileob)
fileob.close()

filed = open("dict.pkl","rb")
dictionary = pickle.load(filed)
filed.close()


In [3]:
def todict(topics):
  dict = {}
  for j in range(len(topics)):
    dict[topics[j][0]] = topics[j][1]
  return dict

In [4]:
def similarity(li1,li2):
    return np.dot(li1,li2)/(np.linalg.norm(li1)*np.linalg.norm(li2))

In [5]:
def make_bigram(bigram,text):
    return [bigram[word] for word in text]

In [39]:
def recommand_news():
    text = input("ENTER ARTICLE :- ")
    
    tokenized = NLTKTokenizer().word_tokenize(text)
    
    tokens = token.copy()
    tokens.append(tokenized)
    
    bigram_phrases = Phrases(token,min_count = 5,threshold = 50)#7

    bigram = Phraser(bigram_phrases)#8

    tokenize_text = make_bigram(bigram,tokens)#9
    
    dtms = [dictionary.doc2bow(word) for word in tokenize_text]
    topics = list(lda[dtms])
    
    length = len(topics)-1
    dic = todict(topics[length])
    
    predict_topic = dict({})#1
    for key in dic.keys():
        if dic[key] >= 0.05:
            predict_topic[key] = dic[key]  
            
    topic_li = [todict(topic) for topic in topics[:-1]]#2
    
    c = 0
    divid = 1
    while c == 0:
        rl = []#3
        rt = []
        for i in range(len(topic_li)):
            l = len(predict_topic.keys())
            d = dict({})
            for j in topic_li[i].keys():
                if j in predict_topic.keys() and topic_li[i][j] >= predict_topic[j]/divid:
                    l-=1
                    d[j] = topic_li[i][j]
            if l == 0:
                rl.append(i)
                rt.append(similarity(list(predict_topic.values()),list(d.values())))
        
        if len(rt) < 10:
            c = 0
            divid+=0.1
        else:
            c = 1
        print(divid)
        
    rlist = list(np.argsort(rt))#4
    rlist.reverse()

    print(predict_topic)
    o = 0
    for simil in rlist[:10]:
        print(f"{o+1:2} => {rt[simil]}")
        o+=1
    return [str(topic_li[rl[i]])+"\n"+df.iloc[rl[i]].article for i in rlist[:10]]

In [40]:
while True:

  rec = recommand_news()

  i=1
  for article in rec:
    print(f"ARTICLE - {i} => ")
    print("=====================================================")
    print(article)
    print("=====================================================")
    i+=1
    if i > 10:
      break

  while True:
    print("\n\n-------------------------------------------------")
    ch = input("DO YOU WANT TO CHECK FOR ANOTHER ARTICLE (Y/N) :- ")
    
    if ch == 'N' or ch == 'Y' or ch == 'n' or ch == 'y':
      break
    else:
      print("PLEASE CHOOSE CORRECTLY......")
    print("-------------------------------------------------\n\n")
  if ch == 'N' or ch == 'n':
    break

ENTER ARTICLE :- "দাদাগিরি'-র (Dadagiri) মঞ্চে রঙের ছোঁয়া। দোলে আগে 'দাদাগিরি'-র মঞ্চে পালিত হল রঙের উৎসব। রঙিন পোশাকে সেজে উঠলেন সৌরভ গঙ্গোপাধ্যায় (Saurav Ganguly)। মঞ্চে হাজির ছোট থেকে বড় পর্দার একাধিক তারকা। নাচে গানে মেতে উঠল মঞ্চ। আনন্দের রঙে মিশলেন সবাই।   হোলির বিশেষ এই পর্বের নাম দেওয়া হয়েছি 'রাঙিয়ে দিয়ে যাও'। মঞ্চে এদিন হাজির ছিলেন মনামী ঘোষ (Monami Ghosh), ঐন্দ্রিলা সেন (Oindrila Sen), পূজা বন্দ্যোপাধ্যায় (Puja Banerjee), শ্রাবন্তী চট্টোপাধ্য়ায়রা (Shrabanti Chatterjee)। নায়িকাদের সঙ্গে নাচের তালে পা মিলিয়েছেন 'দাদা'-ও। চ্যানেলের তরফ থেকে সেই ছবি শেয়ার করে নেওয়া হয়েছে।   এর আগে 'দাদাগিরি'-র মঞ্চে হাজির হয়েছিলেন ভুবন বাদ্যকর। সেখানে দেখা গিয়েছিল, দাদাগিরির মঞ্চে নিজস্ব মহিমায় ভুবন। হাতে রইল বাদামের ঝুড়িও। নিজের প্রতিভায় সবাইকে হারিয়ে শেষ হাসি হেসেছেন ভুবনই। ‘দাদা’র হাত থেকে যখন ট্রফি নিয়েছেন, তখন হাততালিতে ফেটে পড়েছে প্রতিযোগিতার মঞ্চ।  ভুবন সাদামাটা মানুষ, তাঁর কথায় বার্তায় শহুরে চাকচিক্য নেই। সেভাবেই তাঁকে তুলে ধরা হয়েছিল 'দাদাগিরি'-র মঞ্চে। আলাদা করে শেখানো হয়নি কিছুই। তা

DO YOU WANT TO CHECK FOR ANOTHER ARTICLE (Y/N) :- y
ENTER ARTICLE :- "বেঙ্গালুরুতে দিন-রাতের টেস্টে প্রথম ইনিংসে ৫ ও দ্বিতীয় ইনিংসে ৩ উইকেট নিয়েছেন ভারতীয় পেসার যশপ্রীত বুমরা। এই ৮ উইকেটের সৌজন্যে আইসিসি টেস্ট বোলারদের তালিকায় উন্নতি হয়েছে তাঁর। ১০ নম্বর থেকে এক লাফে ৪ নম্বরে উঠে এসেছেন বুমরা। তাঁর পয়েন্ট ৮৩০।  টেস্ট বোলারদের তালিকায় শীর্ষে রয়েছেন অস্ট্রেলিয়ার অধিনায়ক প্যাট কামিন্স। তাঁর পয়েন্ট ৮৯২। দ্বিতীয় স্থানে রয়েছেন বুমরার সতীর্থ রবিচন্দ্রন অশ্বিন। শ্রীলঙ্কার বিরুদ্ধে দু’টেস্টেই ভাল বল করায় তালিকায় উন্নতি হয়েছে তাঁর। অশ্বিনের পয়েন্ট ৮৫০। তিন নম্বরে রয়েছেন দক্ষিণ আফ্রিকার পেসার কাগিসো রাবাডা। তিনি পেয়েছেন ৮৩৫ পয়েন্ট। পাঁচ নম্বরে থাকা পাকিস্তানের পেসার শাহিন শাহ আফ্রিদির পয়েন্ট ৮২২।"
1.1
1.2000000000000002
1.2000000000000002
{6: 0.09717078, 11: 0.8484751}
 1 => 0.9999986886978149
 2 => 0.9999967813491821
 3 => 0.999983012676239
 4 => 0.9999637007713318
 5 => 0.999891459941864
 6 => 0.9998912215232849
 7 => 0.9998791813850403
 8 => 0.9997681379318237
 9 => 0.9996834397315979
10 => 0.

DO YOU WANT TO CHECK FOR ANOTHER ARTICLE (Y/N) :- y
ENTER ARTICLE :- "পাঁচটি রাজ্যে সদ্যসমাপ্ত বিধানসভা নির্বাচনে (Assembly Election) বিজেপি-র (BJP) যে প্রার্থীরা হেরে গিয়েছেন, তাঁদের হারের দায় নিলেন নরেন্দ্র মোদি (Narendra Modi)। বিজেপি-র সংসদীয় দলের বৈঠকে তিনি বলেন, ‘কারও প্রার্থীপদ যদি খারিজ হয়ে গিয়ে থাকে, তাহলে এর দায়িত্ব আমার।’  এই বৈঠকে পরিবারন্ত্রের (Dynasty politics) বিরুদ্ধেও সরব হন মোদি। তিনি দলকে কড়া বার্তা দেওয়ার পাশাপাশি নাম না করে কংগ্রেস (Congress) ও সমাজবাদী পার্টিকে (Samajwadi Party) আক্রমণ করে বলেন, ‘পরিবারতন্ত্র গণতন্ত্রের পক্ষে বিপজ্জনক। আমাদের এর বিরুদ্ধে লড়াই করতে হবে। আমাদের দলে পরিবারতন্ত্র বরদাস্ত করা হবে না। আমার জন্যই বিধানসভা নির্বাচনে দলীয় সাংসদদের ছেলে-মেয়েরা প্রার্থী হতে পারেননি। আমাদের দলে যেমন পরিবারতন্ত্র কায়েম হতে দেওয়া যাবে না, তেমনই যে দলগুলি পরিবারতন্ত্রের ভিত্তিতে চলে, সেই দলগুলির বিরুদ্ধে আমাদের লড়াই চালিয়ে যেতে হবে। পাঁচটি রাজ্যে বিধানসভা নির্বাচনে মানুষ পরিবারতন্ত্রের বিরুদ্ধে রায় দিয়েছেন।’  এই বৈঠকে মোদি ছাড়াও উপস্থিত ছিলেন স্বরাষ্ট্রমন্ত্

DO YOU WANT TO CHECK FOR ANOTHER ARTICLE (Y/N) :- n
